In [2]:
import matplotlib.pyplot as plt

import numpy as np
from scipy import integrate
from scipy.spatial import Delaunay
from scipy.spatial import ConvexHull
from munch import Munch
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

def prism(s, h, x_offset=0, y_offset=0, z_offset=0):
    pts = np.array([
        [0, 0, 0],
        [s/2, s*np.sqrt(3)/2, 0],
        [-s/2, s*np.sqrt(3)/2, 0],
        [0, 0, h],
        [-s/2, s*np.sqrt(3)/2, h],
        [s/2, s*np.sqrt(3)/2, h],
        ])
    x, y, z = pts.T
    x = x + x_offset
    y = y + y_offset
    z = z + z_offset
    ch = ConvexHull(pts)
    return Munch(x=x, y=y, z=z, triangles=ch.simplices, s=s, h=h, max_dim = max(s, h), offset=np.array([x_offset, y_offset, z_offset]))

def prism_mesh(prism):
    return go.Mesh3d(x=prism.x, y=prism.y, z=prism.z, i=prism.triangles[:,0], j=prism.triangles[:,1], k=prism.triangles[:,2])

In [3]:
out = prism(5*4, 5) # contains x, y, z
# Mesh of the prism
# Plot using plotly Mesh3d
T = 5 # thickness

# Three different aspect ratios across the range of prism sizes synthesized.

fig = go.Figure(data=[prism_mesh(x) for x in [prism(T*10, T), prism(T*7,T, y_offset=4, z_offset=10), prism(T*4, T,y_offset=8, z_offset=20)]])

fig.update_layout(scene_zaxis_range=[-2, 30])

fig.show()
# Export figure to html
# fig.write_html("prisms-output.html")

In [17]:
# Draw a sphere and plot it using plotly Mesh3d
# https://plotly.com/python/mesh3d/



fig = go.Figure()
fig.add_trace(go.Scatter3d(
        mode='markers',
        x=[0],
        y=[0],
        z=[0],
        marker=dict(color=px.colors.qualitative.D3, size=[15],sizemode='diameter'
        )
    )
)

fig.show()

In [14]:
# Draw a cylinder and plot it using plotly Mesh3d
# https://stackoverflow.com/questions/44865023/how-to-generate-a-cylinder-in-python

def cylinder(r, h, x_offset=0, y_offset=0, z_offset = 0, n=100):
    theta = np.linspace(0, 2*np.pi, n)
    x = r*np.cos(theta)
    y = r*np.sin(theta)
    z = np.linspace(0, h, n)
    x = np.tile(x, n)
    y = np.tile(y, n)
    z = np.repeat(z, n)
    x = x + x_offset
    y = y + y_offset
    z = z + z_offset
    ch = ConvexHull(np.array([x, y, z]).T)
    return Munch(x=x, y=y, z=z, triangles=ch.simplices, r=r, h=h, max_dim = max(r, h))

# Plot using plotly Mesh3d
fig = go.Figure(data=[prism_mesh(x) for x in [cylinder(T, T*10, x_offset=-8, y_offset=-8), cylinder(T,T*7), cylinder(T, T*4, y_offset=8, x_offset=8)]]
                )


Z = 50
# set x axis limits to -10, 10
R_lim = Z/2
fig.update_layout(scene_xaxis_range=[-R_lim, R_lim], scene_yaxis_range=[-R_lim, R_lim])

# set aspect ratio to 1:1:1
fig.update_layout(scene_aspectmode='manual', scene_aspectratio=dict(x=1, y=1, z=1))


fig.show()